<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/11_fine_tuning_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Fine-Tuning GPT-2

Predicting the next element in a sequence is exactly what a Transformer decoder
does, so it should be no surprise that GPT-2 is actually a Transformer decoder.



We’ll start our NLP journey by following the steps of Alice and Dorothy, from
[Alice’s Adventures in Wonderland](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1476) by Lewis Carroll and [The Wonderful Wizard of Oz](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1740) by L. Frank Baum.

![](https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/images/alice_dorothy.png?raw=1)

*Left: "Alice and the Baby Pig" illustration by John Tenniel's, from "Alice's Adventure's in Wonderland" (1865).*

*Right: "Dorothy meets the Cowardly Lion" illustration by W.W. Denslow, from "The Wonderful Wizard of Oz" (1900)*


##Setup

In [1]:
try:
    import google.colab
    import requests
    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'
    r = requests.get(url, allow_redirects=True)
    open('config.py', 'wb').write(r.content)
except ModuleNotFoundError:
    pass

from config import *
config_chapter11()
# This is needed to render the plots in this chapter
from plots.chapter11 import *

Finished!


In [3]:
%%capture

!pip install accelerate -U
!pip install datasets
!pip -q install spacy
!python -m spacy download en_core_web_sm

In [4]:
import os
import json
import errno
import requests
import numpy as np
from copy import deepcopy
from operator import itemgetter

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

from data_generation.nlp import ALICE_URL, WIZARD_URL, download_text
from stepbystep.v4 import StepByStep
# These are the classes we built in Chapter 10
from seq2seq import *

import spacy
import nltk
from nltk.tokenize import sent_tokenize

In [5]:
from datasets import load_dataset, Split
from transformers import (
    DataCollatorForLanguageModeling,
    BertModel, BertTokenizer, BertForSequenceClassification,
    DistilBertModel, DistilBertTokenizer,
    DistilBertForSequenceClassification,
    AutoModelForSequenceClassification,
    AutoModel, AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, pipeline, TextClassificationPipeline
)
from transformers.pipelines import SUPPORTED_TASKS

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Downloading Books

In [ ]:
!rm -rf data

In [ ]:
# let's download data
HOME_DIR = "data"
download_text(ALICE_URL, HOME_DIR)
download_text(WIZARD_URL, HOME_DIR)

In [ ]:
# let's see the downloaded data
#!cat data/alice28-1476.txt

In [ ]:
#!cat data/wizoz10-1740.txt

We need to remove these additions to the original texts:

In [ ]:
alice_file = os.path.join(HOME_DIR, "alice28-1476.txt")
with open(alice_file, "r") as f:
  # The actual texts of the books are contained between lines 105 and 3703
  alice_text = "".join(f.readlines()[104:3704])

wizard_file = os.path.join(HOME_DIR, "wizoz10-1740.txt")
with open(wizard_file, "r") as f:
  # The actual texts of the books are contained between lines 309 and 5099
  wizard_text = "".join(f.readlines()[310:5100])

In [ ]:
print(alice_text[:500])
print("\n", "#"*70, "\n")
print(wizard_text[:500])

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.8




                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `w

 ###################################################################### 

                    THE WONDERFUL WIZARD OF OZ


                          1.  The Cyclone


    Dorothy lived in the midst of the great Kansas prairies, with
Uncle Henry, who was a farmer, and Aunt Em, who was the farmer's
wife.  Their house was small, for the lumber to build it had to be
carried by wagon many miles.  There were four walls, a floor and a
roof, which made one room; and this room contained a rusty looking

We can partially automate the removal of the extra lines by setting the real start and end lines of each text in a configuration file.

In [ ]:
text_cfg = """fname,start,end
alice28-1476.txt,104,3704
wizoz10-1740.txt,310,5100"""
bytes_written = open(os.path.join(HOME_DIR, 'lines.cfg'), 'w').write(text_cfg)

##Sentence Tokenization

A token is a piece of a text, and to tokenize a text means to split
it into pieces; that is, into a list of tokens.

The most common kind of piece is a word.

So, tokenizing a text usually means to
split it into words using the white space as a separator.

In [ ]:
sentence = "I'm following the white rabbit"
tokens = sentence.split(" ")
tokens

["I'm", 'following', 'the', 'white', 'rabbit']

Let's do sentence tokenization, which means to split a text into its sentences.

In [ ]:
corpus_alice = sent_tokenize(alice_text)
corpus_wizard = sent_tokenize(wizard_text)

len(corpus_alice), (len(corpus_wizard))

(1612, 2240)

Let’s check one sentence from the first corpus of text.

In [ ]:
corpus_alice[2]

'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!'

Let’s check one sentence from the second corpus of text.

In [ ]:
corpus_wizard[30]

'"There\'s a cyclone coming, Em," he called to his wife.'

Our dataset is going to be a collection of CSV files, one file for each book, with each
CSV file containing one sentence per line.

Therefore, we need to:

* clean the line breaks to make sure each sentence is on one line only;
* define an appropriate quote char to "wrap" the sentence such that the original commas and semicolons in the original text do not get misinterpreted as separation chars of the CSV file; and
* add a second column to the CSV file to
identify the original source of the sentence since we’ll be concatenating, and
shuffling the sentences before training a model on our corpora.

The sentence above should end up looking like this:
```log
\"There's a cyclone coming, Em," he called to his wife.\,wizoz10-1740.txt
```

The function below does the grunt work of cleaning, splitting, and saving the
sentences to a CSV file for us:

In [ ]:
def sentence_tokenize(source, quote_char="\\", sep_char=",", include_header=True, include_source=True, extensions=("txt"), **kwargs):
  # If source is a folder, goes through all files inside it that match the desired extensions ('txt' by default)
  if os.path.isdir(source):
    filenames = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f)) and os.path.splitext(f)[1][1:] in extensions]
  elif isinstance(source, str):
    filenames = [source]

  # If there is a configuration file, builds a dictionary with the corresponding start and end lines of each text file
  config_file = os.path.join(source, "lines.cfg")
  config = {}
  if os.path.exists(config_file):
    with open(config_file, "r") as f:
      rows = f.readlines()
    for r in rows[1:]:
      fname, start, end = r.strip().split(",")
      config.update({fname: (int(start), int(end))})

  new_fnames = []
  # For each file of text
  for fname in filenames:
    # If there's a start and end line for that file, use it
    try:
        start, end = config[fname]
    except KeyError:
        start = None
        end = None

    # Opens the file, slices the configures lines (if any)
    # cleans line breaks and uses the sentence tokenizer
    with open(os.path.join(source, fname), 'r') as f:
        contents = (''.join(f.readlines()[slice(start, end, None)]).replace('\n', ' ').replace('\r', ''))
    corpus = sent_tokenize(contents, **kwargs)

    # Builds a CSV file containing tokenized sentences
    base = os.path.splitext(fname)[0]
    new_fname = f'{base}.sent.csv'
    new_fname = os.path.join(source, new_fname)
    with open(new_fname, 'w') as f:
        # Header of the file
        if include_header:
            if include_source:
                f.write('sentence,source\n')
            else:
                f.write('sentence\n')
        # Writes one line for each sentence
        for sentence in corpus:
            if include_source:
                f.write(f'{quote_char}{sentence}{quote_char}{sep_char}{fname}\n')
            else:
                f.write(f'{quote_char}{sentence}{quote_char}\n')
    new_fnames.append(new_fname)

  # Returns list of the newly generated CSV files
  return sorted(new_fnames)

In [ ]:
new_fnames = sentence_tokenize(HOME_DIR)
new_fnames

['data/alice28-1476.sent.csv', 'data/wizoz10-1740.sent.csv']

##Spacy sentence tokenization

In [ ]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

sentences = []
for doc in nlp.pipe(corpus_alice):
  sentences.extend(sent.text for sent in doc.sents)

len(sentences), sentences[2]

(1615,
 'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!')

##HuggingFace’s Dataset

In [ ]:
# let's load from local files using HuggingFace
dataset = load_dataset(path="csv", data_files=new_fnames, quotechar="\\", split=Split.TRAIN)

In [ ]:
# let's see attributes, like features, num_columns, and shape
dataset.features, dataset.num_columns, dataset.shape

({'sentence': Value(dtype='string', id=None),
  'source': Value(dtype='string', id=None)},
 2,
 (3852, 2))

In [ ]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt'}

In [ ]:
dataset["sentence"][:3]

["                ALICE'S ADVENTURES IN WONDERLAND                            Lewis Carroll                 THE MILLENNIUM FULCRUM EDITION 2.8                                 CHAPTER I                        Down the Rabbit-Hole     Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:  once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'",
 'So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!']

In [ ]:
dataset["source"][:3]

['alice28-1476.txt', 'alice28-1476.txt', 'alice28-1476.txt']

In [ ]:
# check the unique sources
dataset.unique("source")

['alice28-1476.txt', 'wizoz10-1740.txt']

In [ ]:
# let's create new columns
def is_alice_label(row):
  is_alice = int(row["source"] == "alice28-1476.txt")
  return {"label": is_alice}

In [ ]:
dataset = dataset.map(is_alice_label)

In [ ]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt',
 'label': 1}

In [ ]:
# Now, we can finally shuffle the dataset and split it into training and test sets
shuffled_dataset = dataset.shuffle(seed=42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 3081
    })
    test: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 771
    })
})

In [ ]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [ ]:
train_dataset[0]

{'sentence': 'They all sat down at once, in a large ring, with the Mouse in the middle.',
 'source': 'alice28-1476.txt',
 'label': 1}

##Pipeline

In [ ]:
# Let’s load the GPT-2-based text generation pipeline:
text_generator = pipeline("text-generation")

Then, let’s use the first two paragraphs from Alice’s Adventures in Wonderland as
our base text:

In [11]:
base_text = """
Alice was beginning to get very tired of sitting by her sister on
the bank, and of having nothing to do: once or twice she had peeped
into the book her sister was reading, but it had no pictures or
conversations in it, `and what is the use of a book,'thought Alice
`without pictures or conversation?' So she was considering in her
own mind (as well as she could, for the hot day made her feel very
sleepy and stupid), whether the pleasure of making a daisy-chain
would be worth the trouble of getting up and picking the daisies,
when suddenly a White Rabbit with pink eyes ran close by her.
"""

In [9]:
text_generator.model.config.task_specific_params

{'text-generation': {'do_sample': True, 'max_length': 50}}

In [12]:
result = text_generator(base_text, max_length=250)
print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Alice was beginning to get very tired of sitting by her sister on
the bank, and of having nothing to do: once or twice she had peeped
into the book her sister was reading, but it had no pictures or
conversations in it, `and what is the use of a book,'thought Alice
`without pictures or conversation?' So she was considering in her
own mind (as well as she could, for the hot day made her feel very
sleepy and stupid), whether the pleasure of making a daisy-chain
would be worth the trouble of getting up and picking the daisies,
when suddenly a White Rabbit with pink eyes ran close by her.
As the child had walked into it, her eyes flashed
to a white rabbit sitting on the floor, holding a large daisy-chain
and his wife, on each thigh.
"What do they
now-"
'that we should be here when the
blubber comes out--to talk here? or at least
here'
"they came to talk to me and
I heard that some of my friends went
after them. As they came they did not do any
something except


By the way, if you try using greedy decoding instead (setting `do_sample=False`), the generated text simply and annoyingly repeats the same text over and over again:

In [13]:
result = text_generator(base_text, max_length=250, do_sample=False)
print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Alice was beginning to get very tired of sitting by her sister on
the bank, and of having nothing to do: once or twice she had peeped
into the book her sister was reading, but it had no pictures or
conversations in it, `and what is the use of a book,'thought Alice
`without pictures or conversation?' So she was considering in her
own mind (as well as she could, for the hot day made her feel very
sleepy and stupid), whether the pleasure of making a daisy-chain
would be worth the trouble of getting up and picking the daisies,
when suddenly a White Rabbit with pink eyes ran close by her.
'Oh, my dear, I am so glad to see you!' said Alice, 'I am so glad to see you!'
'Oh, my dear, I am so glad to see you!' said Alice, 'I am so glad to see you!'
'Oh, my dear, I am so glad to see you!' said Alice, 'I am so glad to see you!'
'Oh, my dear, I am so glad to see you!' said Alice, 'I am so glad to see


Wait a minute! Aren’t we fine-tuning GPT-2 so it can write text in a
given style?



##Data Preparation

In order to capture the style of Lewis Carroll’s Alice’s Adventures in Wonderland, we
need to use a dataset containing sentences from that book alone.

In [ ]:
# Let’s create our first real BERT tokenizer
auto_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize(row):
  return auto_tokenizer(
      row["sentence"],
      truncation=True,
      padding="max_length",
      max_length=30
  )

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
print(tokenized_train_dataset[0])

{'sentence': 'They all sat down at once, in a large ring, with the Mouse in the middle.', 'source': 'alice28-1476.txt', 'label': 1, 'input_ids': [101, 2027, 2035, 2938, 2091, 2012, 2320, 1010, 1999, 1037, 2312, 3614, 1010, 2007, 1996, 8000, 1999, 1996, 2690, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
# But we don’t need all these columns for training; we need only the first three.
tokenized_train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

tokenized_test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

In [ ]:
tokenized_train_dataset[0]

{'label': tensor(1),
 'input_ids': tensor([ 101, 2027, 2035, 2938, 2091, 2012, 2320, 1010, 1999, 1037, 2312, 3614,
         1010, 2007, 1996, 8000, 1999, 1996, 2690, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

##Training

In [ ]:
# let’s train model with trainer
trainer = Trainer(
    model=bert_auto_model,
    train_dataset=tokenized_train_dataset)

In [ ]:
# let's see default trainer arguments
trainer.args

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_length=False,
half_precision_backend=au

In [ ]:
# let's override the default trainer arguments
training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,  # means mini-batch has size eight
    evaluation_strategy="steps",
    eval_steps=300,
    logging_steps=300,
    gradient_accumulation_steps=8
)

In [ ]:
# let's define a function that computes the desired metrics, and returns a dictionary
def compute_metrics(eval_pred):
  predictions = eval_pred.predictions
  labels = eval_pred.label_ids
  predictions = np.argmax(predictions, axis=1)
  return {"accuracy": (predictions==labels).mean()}

In [ ]:
# let’s redefine the trainer
trainer = Trainer(
    model=bert_auto_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics)

In [ ]:
# There we go—we’re 100% ready to call the glorious train() method
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
300,0.068800,0.128313,0.974060


TrainOutput(global_step=385, training_loss=0.0771487941989651, metrics={'train_runtime': 1259.9611, 'train_samples_per_second': 2.445, 'train_steps_per_second': 0.306, 'total_flos': 23906225851200.0, 'train_loss': 0.0771487941989651, 'epoch': 1.0})

In [ ]:
# let's check the final validation
trainer.evaluate()

{'eval_loss': 0.10381420701742172,
 'eval_accuracy': 0.9792477302204928,
 'eval_runtime': 38.0468,
 'eval_samples_per_second': 20.265,
 'eval_steps_per_second': 2.549,
 'epoch': 1.0}

In [ ]:
!rm -rf bert_alice_vs_wizard

In [ ]:
# let's save it to disk
trainer.save_model("bert_alice_vs_wizard")

os.listdir("bert_alice_vs_wizard")

['training_args.bin', 'model.safetensors', 'config.json']

In [ ]:
# Later on, we can easily load the trained model
loaded_model = (AutoModelForSequenceClassification.from_pretrained("bert_alice_vs_wizard"))

loaded_model.device

device(type='cpu')

The model is loaded to the CPU by default, but we can send it to a different device.

In [ ]:
loaded_model.to(device)
loaded_model.device

device(type='cpu')

##Predictions

In [ ]:
sentence = "Down the yellow brick rabbit hole"
tokens = auto_tokenizer(sentence, return_tensors="pt")

In [ ]:
# we need to make sure the tensors are in the same device as the model.
print(type(tokens))
tokens.to(loaded_model.device)

<class 'transformers.tokenization_utils_base.BatchEncoding'>


{'input_ids': tensor([[  101,  2091,  1996,  3756,  5318, 10442,  4920,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
loaded_model.eval()

logits = loaded_model(input_ids=tokens["input_ids"], attention_mask=tokens["attention_mask"])
logits

SequenceClassifierOutput(loss=None, logits=tensor([[-1.9329,  2.0669]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# The largest logit corresponds to the predicted class as usual
logits.logits.argmax(dim=1)

tensor([1])

BERT has spoken: The sentence "Down the Yellow Brick Rabbit Hole" is more likely
coming from The Wonderful Wizard of Oz (the negative class of our binary
classification task).

##Pipelines

Pipelines can handle all these steps for us, we just have to choose the appropriate one.

Let’s use the `TextClassificationPipeline`

In [ ]:
device_index = (loaded_model.device.index if loaded_model.device.type != "cpu" else -1)

classifier = TextClassificationPipeline(model=loaded_model, tokenizer=auto_tokenizer, device=device_index)

In [ ]:
# Now we can make predictions using the original sentences
classifier(["Down the Yellow Brick Rabbit Hole", "Alice rules!"])

[{'label': 'LABEL_1', 'score': 0.9820101857185364},
 {'label': 'LABEL_1', 'score': 0.9998201727867126}]

The model seems pretty confident that the first sentence is from The Wonderful
Wizard of Oz (negative class) and that the second sentence is from Alice’s
Adventures in Wonderland (positive class).

In [ ]:
# We can make the output a bit more intuitive
loaded_model.config.id2label = {0: "Wizard", 1: "Alice"}

classifier(["Down the Yellow Brick Rabbit Hole", "Alice rules!"])

[{'label': 'Alice', 'score': 0.9820101857185364},
 {'label': 'Alice', 'score': 0.9998201727867126}]